In [460]:
import pandas as pd 
import numpy as np 
import os 
import requests
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import pprint
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics.pairwise import euclidean_distances
from bs4 import BeautifulSoup

In [ ]:
money_puck = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/other/mp_all_skater.csv')
money_puck_goalies = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/goalies/goalies.csv')
nhl_api_goalies = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/goalies.csv')
name_playerid_goalie = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/goalies/name_id_goalie.csv')
name_playerid = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/other/name_playerId.csv')
salaries = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/salary/cap_all.csv')
salaries_goalies = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/salary/goalies_cap.csv')
goalies_quant = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/master_copies/goalie_quant.csv')

In [462]:
# Merge the DataFrames on the player_id column
merged_goalie_result = pd.merge(name_playerid_goalie, nhl_api_goalies, on='player_id', how='outer')

# Save the merged DataFrame to a CSV file if needed
#merged_goalie_result.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/other/merged_goalie_result.csv', index=False)
merged_goalie_result = merged_goalie_result.drop(columns=['Unnamed: 0'])
len(merged_goalie_result)

321

In [463]:
merged_goalie_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       318 non-null    object
 1   player_id  321 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.1+ KB


In [464]:
# Drop rows where the 'name' column is null
merged_goalie_result = merged_goalie_result.dropna(subset=['name'])

In [465]:
#merged_goalie_result = merged_goalie_result.drop(columns=['Unnamed: 0'])

# Split the name_x column into first_name and last_name
def split_name(name):
    parts = name.split()
    if len(parts) > 1:
        return parts[-1], ' '.join(parts[:-1])
    else:
        return parts[0], ''

merged_goalie_result[['last_name', 'first_name']] = merged_goalie_result['name'].apply(lambda x: pd.Series(split_name(x)))

# Combine last_name and first_name into a single column with the format "Last, First"
merged_goalie_result['name'] = merged_goalie_result['last_name'] + ', ' + merged_goalie_result['first_name']

# Drop the temporary first_name and last_name columns
merged_goalie_result = merged_goalie_result.drop(columns=['first_name', 'last_name'])

# Display the first few rows of the DataFrame to verify the change
merged_goalie_result



,name,player_id
0,"Joseph, Curtis",8448382
1,"Kolzig, Olie",8448535
2,"Brodeur, Martin",8455710
3,"Osgood, Chris",8458568
4,"Fernandez, Manny",8458988
...,...,...
315,"Berlin, Matthew",8483158
316,"Alexander, Jett",8483183
317,"Stauber, Jaxson",8483530
319,"Perets, Yaniv",8484293


In [466]:
money_puck_goalies.columns

Index(['Unnamed: 0', 'name', 'icetime', 'x_goals', 'goals',
       'unblocked_shot_attempts', 'x_rebounds', 'rebounds', 'x_freeze',
       'freeze', 'x_on_goal', 'on_goal', 'x_play_stopped', 'play_stopped',
       'x_play_continued_in_zone', 'play_continued_in_zone',
       'x_play_continued_outside_zone', 'play_continued_outside_zone',
       'flurry_adjusted_x_goals', 'low_danger_shots', 'medium_danger_shots',
       'high_danger_shots', 'low_danger_x_goals', 'medium_danger_x_goals',
       'high_danger_x_goals', 'low_danger_goals', 'medium_danger_goals',
       'high_danger_goals', 'blocked_shot_attempts', 'penalty_minutes',
       'penalties'],
      dtype='object')

In [467]:
money_puck_goalies = money_puck_goalies.drop(columns=['Unnamed: 0', 'x_goals', 'x_rebounds', 'x_freeze'
                                                      ,'x_on_goal', 'x_play_stopped', 'x_play_continued_in_zone'
                                                      , 'x_play_continued_outside_zone', 'flurry_adjusted_x_goals'
                                                      , 'low_danger_x_goals', 'medium_danger_x_goals'
                                                      , 'high_danger_x_goals', 'penalty_minutes','penalties'
       ])
money_puck_goalies.head()

,name,icetime,goals,unblocked_shot_attempts,rebounds,freeze,on_goal,play_stopped,play_continued_in_zone,play_continued_outside_zone,low_danger_shots,medium_danger_shots,high_danger_shots,low_danger_goals,medium_danger_goals,high_danger_goals,blocked_shot_attempts
0,Aaron Dell,3163.584615,2.576923,49.930769,2.153846,5.707692,27.053846,0.807692,11.684615,14.307692,27.069231,7.869231,2.276923,0.807692,1.069231,0.700000,12.715385
1,Adin Hill,3327.886364,2.465909,54.869318,2.568182,6.159091,26.960227,0.545455,13.323864,14.426136,29.909091,7.159091,2.465909,0.931818,0.869318,0.664773,15.335227
2,Akira Schmid,2843.045455,2.250000,42.045455,2.295455,4.704545,22.522727,0.431818,10.727273,11.250000,22.772727,6.727273,2.068182,0.522727,1.204545,0.522727,10.477273
3,Al Montoya,3267.851190,2.410714,49.755952,1.869048,6.315476,26.297619,0.994048,11.071429,14.166667,27.380952,7.071429,2.267857,0.851190,0.946429,0.613095,13.035714
4,Aleksei Kolosov,3132.400000,3.000000,53.933333,2.466667,4.333333,23.133333,0.133333,11.000000,15.600000,26.666667,6.800000,2.933333,1.133333,0.533333,1.333333,17.533333


In [468]:
#money_puck_goalies =money_puck_goalies.drop(columns=['Unnamed: 0'])

# Split the name_x column into first_name and last_name
def split_name(name):
    parts = name.split()
    if len(parts) > 1:
        return parts[-1], ' '.join(parts[:-1])
    else:
        return parts[0], ''

money_puck_goalies[['last_name', 'first_name']] =money_puck_goalies['name'].apply(lambda x: pd.Series(split_name(x)))

# Combine last_name and first_name into a single column with the format "Last, First"
money_puck_goalies['name'] =money_puck_goalies['last_name'] + ', ' +money_puck_goalies['first_name']

# Drop the temporary first_name and last_name columns
money_puck_goalies =money_puck_goalies.drop(columns=['first_name', 'last_name'])

# Display the first few rows of the DataFrame to verify the change
money_puck_goalies

,name,icetime,goals,unblocked_shot_attempts,rebounds,freeze,on_goal,play_stopped,play_continued_in_zone,play_continued_outside_zone,low_danger_shots,medium_danger_shots,high_danger_shots,low_danger_goals,medium_danger_goals,high_danger_goals,blocked_shot_attempts
0,"Dell, Aaron",3163.584615,2.576923,49.930769,2.153846,5.707692,27.053846,0.807692,11.684615,14.307692,27.069231,7.869231,2.276923,0.807692,1.069231,0.700000,12.715385
1,"Hill, Adin",3327.886364,2.465909,54.869318,2.568182,6.159091,26.960227,0.545455,13.323864,14.426136,29.909091,7.159091,2.465909,0.931818,0.869318,0.664773,15.335227
2,"Schmid, Akira",2843.045455,2.250000,42.045455,2.295455,4.704545,22.522727,0.431818,10.727273,11.250000,22.772727,6.727273,2.068182,0.522727,1.204545,0.522727,10.477273
3,"Montoya, Al",3267.851190,2.410714,49.755952,1.869048,6.315476,26.297619,0.994048,11.071429,14.166667,27.380952,7.071429,2.267857,0.851190,0.946429,0.613095,13.035714
4,"Kolosov, Aleksei",3132.400000,3.000000,53.933333,2.466667,4.333333,23.133333,0.133333,11.000000,15.600000,26.666667,6.800000,2.933333,1.133333,0.533333,1.333333,17.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,"Fasth, Viktor",3300.603175,2.555556,52.396825,1.904762,6.444444,26.698413,0.825397,12.190476,14.444444,29.015873,7.603175,2.063492,1.047619,0.761905,0.746032,13.714286
212,"Husso, Ville",3375.676056,2.859155,54.859155,2.661972,5.809859,28.845070,0.612676,13.852113,15.338028,30.669014,7.654930,2.690141,1.246479,0.943662,0.669014,13.845070
213,"Vanecek, Vitek",3353.813187,2.609890,51.648352,2.379121,4.774725,26.912088,0.549451,13.054945,15.016484,28.648352,7.225275,2.510989,0.851648,0.945055,0.813187,13.263736
214,"Danis, Yann",2964.020408,2.306122,48.306122,1.673469,5.653061,25.081633,0.612245,11.244898,13.673469,26.204082,6.816327,2.346939,0.795918,0.693878,0.816327,12.938776


In [469]:
goalies = pd.merge(merged_goalie_result, money_puck_goalies, on='name', how='inner')
goalies

,name,player_id,icetime,goals,unblocked_shot_attempts,rebounds,freeze,on_goal,play_stopped,play_continued_in_zone,play_continued_outside_zone,low_danger_shots,medium_danger_shots,high_danger_shots,low_danger_goals,medium_danger_goals,high_danger_goals,blocked_shot_attempts
0,"Joseph, Curtis",8448382,2507.100000,2.500000,37.400000,1.050000,2.950000,19.150000,0.700000,9.550000,12.050000,20.400000,6.350000,1.900000,0.750000,0.900000,0.850000,8.750000
1,"Brodeur, Martin",8455710,3459.275168,2.288591,43.291946,1.479866,4.567114,25.010067,0.751678,9.533557,14.567114,25.124161,6.134228,1.862416,0.778523,0.828859,0.681208,10.171141
2,"Osgood, Chris",8458568,3407.562500,2.862500,44.487500,1.475000,5.425000,25.837500,1.137500,11.025000,13.387500,25.850000,6.950000,2.275000,1.062500,0.850000,0.950000,9.412500
3,"Fernandez, Manny",8458988,3523.000000,2.535714,48.392857,1.821429,6.750000,28.250000,0.964286,10.535714,15.607143,27.821429,7.464286,2.571429,0.821429,0.821429,0.892857,10.535714
4,"Khabibulin, Nikolai",8459140,3431.680982,2.742331,52.907975,1.981595,4.963190,29.122699,0.748466,13.018405,16.300613,29.687117,7.803681,2.374233,0.987730,0.871166,0.883436,13.042945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,"Daws, Nico",8482076,3082.666667,2.529412,48.450980,2.529412,5.803922,24.960784,0.431373,10.882353,14.431373,26.862745,7.588235,2.294118,1.039216,0.745098,0.745098,11.705882
208,"Askarov, Yaroslav",8482137,3211.125000,2.687500,56.437500,3.437500,4.312500,26.687500,0.062500,14.625000,15.500000,29.750000,7.812500,3.187500,1.062500,0.875000,0.750000,15.687500
209,"Levi, Devon",8482221,3391.615385,3.102564,55.000000,4.256410,5.256410,29.384615,0.179487,13.461538,15.256410,29.435897,8.179487,4.153846,1.205128,1.000000,0.897436,13.230769
210,"Soderblom, Arvid",8482821,3356.585366,3.317073,58.817073,3.134146,5.634146,30.097561,0.231707,13.634146,17.231707,31.829268,8.317073,3.146341,1.390244,1.146341,0.780488,15.524390


In [479]:
goalies.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/linear_reg_csv_files/goalies_all.csv')

In [470]:
goalies_quant = goalies_quant.drop(columns=['Unnamed: 0','Rk'])
goalies_quant = goalies_quant.rename(columns={'Name': 'name'})
# Split the name_x column into first_name and last_name
def split_name(name):
    parts = name.split()
    if len(parts) > 1:
        return parts[-1], ' '.join(parts[:-1])
    else:
        return parts[0], ''

goalies_quant[['last_name', 'first_name']] =goalies_quant['name'].apply(lambda x: pd.Series(split_name(x)))

# Combine last_name and first_name into a single column with the format "Last, First"
goalies_quant['name'] =goalies_quant['last_name'] + ', ' +goalies_quant['first_name']

# Drop the temporary first_name and last_name columns
goalies_quant =goalies_quant.drop(columns=['first_name', 'last_name'])

# Display the first few rows of the DataFrame to verify the change
goalies_quant

,name,Born,GP,GAA,SV%,W,L,GA,SV,SOG,SO,TIME
0,"Fleury, Marc-Andre",1984,1047,2.60,0.912,573,337,2622,27123,29745,76,60531
1,"Bobrovsky, Sergei",1988,745,2.58,0.914,424,240,1849,19745,21594,48,42978
2,"Quick, Jonathan",1986,801,2.49,0.911,402,289,1929,19661,21590,63,46482
3,"Vasilevski, Andrei",1994,529,2.54,0.917,324,162,1316,14510,15825,38,31125
4,"Hellebuyck, Connor",1993,556,2.57,0.918,314,183,1382,15407,16789,43,32326
...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Skarek, Jakub",1999,2,3.94,0.872,0,1,5,34,39,0,76
91,"Miner, Trent",2001,2,2.62,0.879,0,1,4,29,33,0,92
92,"Commesso, Drew",2002,2,3.36,0.846,0,1,4,22,26,0,71
93,"Perets, Yaniv",2000,2,2.86,0.875,0,0,1,7,8,0,21


In [471]:
goalie = pd.merge(goalies_quant, goalies, on='name', how='inner')
goalie

,name,Born,GP,GAA,SV%,W,L,GA,SV,SOG,...,play_stopped,play_continued_in_zone,play_continued_outside_zone,low_danger_shots,medium_danger_shots,high_danger_shots,low_danger_goals,medium_danger_goals,high_danger_goals,blocked_shot_attempts
0,"Fleury, Marc-Andre",1984,1047,2.60,0.912,573,337,2622,27123,29745,...,1.106529,12.774341,15.033219,29.471936,7.162658,2.344788,0.817869,0.876289,0.761741,13.938144
1,"Bobrovsky, Sergei",1988,745,2.58,0.914,424,240,1849,19745,21594,...,1.185235,12.524832,14.711409,29.585235,7.465772,2.575839,0.859060,0.885906,0.739597,12.987919
2,"Quick, Jonathan",1986,801,2.49,0.911,402,289,1929,19661,21590,...,0.929825,12.563910,14.701754,29.071429,7.274436,2.343358,0.840852,0.867168,0.700501,12.233083
3,"Hellebuyck, Connor",1993,556,2.57,0.918,314,183,1382,15407,16789,...,0.677477,13.949550,15.920721,31.396396,7.879279,2.821622,0.794595,0.877477,0.812613,13.920721
4,"Andersen, Frederik",1989,509,2.55,0.916,304,132,1248,13554,14802,...,1.009823,13.589391,15.825147,30.905697,7.711198,2.430255,0.840864,0.872299,0.746562,12.687623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,"Primeau, Cayden",1999,55,3.69,0.884,13,24,174,1328,1502,...,0.200000,12.818182,13.963636,28.945455,7.345455,2.945455,1.236364,1.181818,0.745455,14.400000
65,"Silovs, Arturs",2001,19,3.13,0.880,8,8,57,417,474,...,0.210526,12.157895,12.947368,28.684211,5.473684,2.789474,1.526316,0.578947,0.894737,14.263158
66,"Greaves, Jet",2001,15,3.28,0.910,5,8,48,487,535,...,0.133333,17.000000,18.000000,37.400000,9.533333,4.200000,1.133333,1.533333,0.533333,17.133333
67,"Askarov, Yaroslav",2002,16,3.01,0.899,5,7,43,384,427,...,0.062500,14.625000,15.500000,29.750000,7.812500,3.187500,1.062500,0.875000,0.750000,15.687500


In [472]:
goalie.columns.to_list()

['name',
 'Born',
 'GP',
 'GAA',
 'SV%',
 'W',
 'L',
 'GA',
 'SV',
 'SOG',
 'SO',
 'TIME',
 'player_id',
 'icetime',
 'goals',
 'unblocked_shot_attempts',
 'rebounds',
 'freeze',
 'on_goal',
 'play_stopped',
 'play_continued_in_zone',
 'play_continued_outside_zone',
 'low_danger_shots',
 'medium_danger_shots',
 'high_danger_shots',
 'low_danger_goals',
 'medium_danger_goals',
 'high_danger_goals',
 'blocked_shot_attempts']

In [473]:
goalie = goalie.drop(columns=['icetime',
 'goals',
 'unblocked_shot_attempts',
 'rebounds',
 'freeze',
 'on_goal',
 'play_stopped',
 'play_continued_in_zone',
 'play_continued_outside_zone',
 'low_danger_shots',
 'medium_danger_shots',
 'high_danger_shots',
 'low_danger_goals',
 'medium_danger_goals',
 'high_danger_goals',
 'blocked_shot_attempts',
 'Born', 'GAA', 'SV'])

In [474]:
salaries_goalies = salaries_goalies.rename(columns={'PLAYERS': 'name'})

In [475]:
goalies_salaries = pd.merge(goalie, salaries_goalies, on='name', how='outer')
#goalies_salaries = goalies_salaries.drop(columns=['TOTAL', 'TERM', 'TYPE', 'TEAM', 'POS', 'HAND', 'AGE', 'G', 'A', 'PTS'
                                              #, 'SV%'])

goalies_salaries

,name,GP,SV%_x,W,L,GA,SOG,SO,TIME,player_id,...,HAND,AGE,G,A,PTS,SV%_y,CAP HIT,TOTAL,TERM,TYPE
0,"Akhtyamov, Artur",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Left,23.0,0.0,0.0,0.0,0.000,"$851,667","$850,000",3 yrs,ELC
1,"Allen, Jake",454.0,0.908,206.0,181.0,1176.0,12729.0,28.0,25626.0,8474596.0,...,Left,34.0,0.0,0.0,0.0,0.911,"$3,850,000","$3,300,000",2 yrs,Std (Ext)
2,"Andersen, Frederik",509.0,0.916,304.0,132.0,1248.0,14802.0,28.0,29401.0,8475883.0,...,Left,35.0,0.0,1.0,0.0,0.912,"$3,400,000","$3,100,000",2 yrs,Std
3,"Annunen, Justus",44.0,0.901,24.0,14.0,113.0,1142.0,2.0,2375.0,8481020.0,...,Left,25.0,0.0,0.0,0.0,0.872,"$837,500","$800,000",2 yrs,Std (Ext)
4,"Askarov, Yaroslav",16.0,0.899,5.0,7.0,43.0,427.0,0.0,857.0,8482137.0,...,Right,22.0,0.0,1.0,0.0,0.896,"$925,000","$925,000",3 yrs,ELC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,"Wallstedt, Jesper",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Left,22.0,0.0,0.0,0.0,0.843,"$925,000","$925,000",3 yrs,ELC
119,"Wedgewood, Scott",149.0,0.906,57.0,54.0,404.0,4296.0,8.0,8306.0,8475809.0,...,Left,32.0,0.0,0.0,0.0,0.878,"$1,500,000","$1,500,000",2 yrs,Std
120,"Wolf, Dustin",58.0,0.909,30.0,20.0,153.0,1677.0,3.0,3419.0,8481692.0,...,Left,23.0,0.0,3.0,0.0,0.914,"$850,000","$850,000",2 yrs,Std (2-way)
121,"Woll, Joseph",70.0,0.909,43.0,25.0,189.0,2072.0,2.0,4093.0,8479361.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [477]:
# Check out who is in the NaN rows of the goalies_salaries df to see if they are important
nan_goalies = goalies_salaries[goalies_salaries['player_id'].isnull()]
# Drop rows from goalies_salaries that are present in nan_goalies
goalies_salaries = goalies_salaries.drop(nan_goalies.index)
goalies_salaries


,name,GP,SV%_x,W,L,GA,SOG,SO,TIME,player_id,...,HAND,AGE,G,A,PTS,SV%_y,CAP HIT,TOTAL,TERM,TYPE
1,"Allen, Jake",454.0,0.908,206.0,181.0,1176.0,12729.0,28.0,25626.0,8474596.0,...,Left,34.0,0.0,0.0,0.0,0.911,"$3,850,000","$3,300,000",2 yrs,Std (Ext)
2,"Andersen, Frederik",509.0,0.916,304.0,132.0,1248.0,14802.0,28.0,29401.0,8475883.0,...,Left,35.0,0.0,1.0,0.0,0.912,"$3,400,000","$3,100,000",2 yrs,Std
3,"Annunen, Justus",44.0,0.901,24.0,14.0,113.0,1142.0,2.0,2375.0,8481020.0,...,Left,25.0,0.0,0.0,0.0,0.872,"$837,500","$800,000",2 yrs,Std (Ext)
4,"Askarov, Yaroslav",16.0,0.899,5.0,7.0,43.0,427.0,0.0,857.0,8482137.0,...,Right,22.0,0.0,1.0,0.0,0.896,"$925,000","$925,000",3 yrs,ELC
5,"Binnington, Jordan",326.0,0.906,165.0,115.0,880.0,9392.0,18.0,18868.0,8476412.0,...,Left,31.0,0.0,0.0,0.0,0.897,"$6,000,000","$7,500,000",6 yrs,Std (Ext)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,"Varlamov, Semyon",621.0,0.916,289.0,232.0,1572.0,18623.0,41.0,35629.0,8473575.0,...,Left,36.0,0.0,0.0,0.0,0.889,"$5,750,000","$2,750,000",4 yrs,35+ Contract
115,"Vejmelka, Karel",183.0,0.900,62.0,92.0,556.0,5573.0,6.0,10286.0,8478872.0,...,Right,28.0,0.0,0.0,0.0,0.908,"$2,725,000","$2,775,000",3 yrs,Std (Ext)
119,"Wedgewood, Scott",149.0,0.906,57.0,54.0,404.0,4296.0,8.0,8306.0,8475809.0,...,Left,32.0,0.0,0.0,0.0,0.878,"$1,500,000","$1,500,000",2 yrs,Std
120,"Wolf, Dustin",58.0,0.909,30.0,20.0,153.0,1677.0,3.0,3419.0,8481692.0,...,Left,23.0,0.0,3.0,0.0,0.914,"$850,000","$850,000",2 yrs,Std (2-way)


In [478]:
goalies_salary.head(50)

,name,GP,SV%_x,W,L,GA,SOG,SO,TIME,CAP HIT
1,"Allen, Jake",454.0,0.908,206.0,181.0,1176.0,12729.0,28.0,25626.0,"$3,850,000"
2,"Andersen, Frederik",509.0,0.916,304.0,132.0,1248.0,14802.0,28.0,29401.0,"$3,400,000"
3,"Annunen, Justus",44.0,0.901,24.0,14.0,113.0,1142.0,2.0,2375.0,"$837,500"
4,"Askarov, Yaroslav",16.0,0.899,5.0,7.0,43.0,427.0,0.0,857.0,"$925,000"
5,"Binnington, Jordan",326.0,0.906,165.0,115.0,880.0,9392.0,18.0,18868.0,"$6,000,000"
9,"Blackwood, Mackenzie",243.0,0.906,99.0,98.0,676.0,7188.0,13.0,13691.0,"$2,350,000"
11,"Bobrovsky, Sergei",745.0,0.914,424.0,240.0,1849.0,21594.0,48.0,42978.0,"$10,000,000"
20,"Comrie, Eric",73.0,0.898,31.0,34.0,216.0,2110.0,3.0,4093.0,NaN
21,"Copley, Pheonix",77.0,0.898,44.0,17.0,202.0,1988.0,4.0,4261.0,NaN
24,"Daccord, Joey",114.0,0.908,44.0,45.0,293.0,3165.0,4.0,6436.0,"$1,200,000"


In [ ]:
goalies_salary.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/linear_reg_csv_files/goalies_lr.csv')

In [ ]:
nhl_api.columns

In [ ]:
nhl_api_grouped =  nhl_api.groupby(by=['player_id']).sum().reset_index()
nhl_api_grouped = nhl_api_grouped.drop(columns=['goals', 'assists', 'points', 'plusMinus', 'shots', 'shifts', 'pim'])
len(nhl_api_grouped)


In [ ]:
# Rename the playerId_all column to player_id
name_playerid = name_playerid.rename(columns={'playerId_all': 'player_id'}).drop(columns=('I_F_goals_all'))
name_playerid

In [ ]:
# Merge the DataFrames on the player_id column
merged_skater_result = pd.merge(name_playerid, nhl_api_grouped, on='player_id', how='inner')

# Save the merged DataFrame to a CSV file if needed
#merged_skater_result.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/other/merged_skater_result.csv', index=False)
len(merged_skater_result)

In [ ]:
money_puck = money_puck.drop(columns=['Unnamed: 0'])
len(money_puck)

In [ ]:
money_puck_merged = pd.merge(money_puck, merged_skater_result, on='name', how='inner')
money_puck_merged

In [ ]:
money_puck_merged = money_puck_merged.drop(columns=(['player_id_x', 'player_id_y', 'gameCount']))

In [ ]:
money_puck_merged.info()

In [ ]:
# # Check if playerId in money_puck_merged is in nhl_api_grouped
# money_puck_merged['playerId_in_nhl_api'] = money_puck_merged['player_id'].isin(nhl_api_grouped['player_id'])

# # Display the rows where playerId is in nhl_api_grouped
# matches = money_puck_merged[money_puck_merged['playerId_in_nhl_api']]

# # Display the matches
# print(matches.head())

# # Check the number of matches
# num_matches = matches.shape[0]
# print(f'Number of matches: {num_matches}')

# # Check the number of unique playerIds in each DataFrame
# num_unique_money_puck = money_puck_merged['player_id'].nunique()
# num_unique_nhl_api = nhl_api_grouped['player_id'].nunique()
# print(f'Number of unique playerIds in money_puck_merged: {num_unique_money_puck}')
# print(f'Number of unique playerIds in nhl_api_grouped: {num_unique_nhl_api}')

In [ ]:
# # Rename the player_id column in nhl_api_grouped to match player_id in money_puck
# nhl_api_grouped = nhl_api_grouped.rename(columns={'player_id': 'player_id'})

# # Merge the DataFrames on the player_id column
# merged_df = pd.merge(nhl_api_grouped, money_puck_merged, on='player_id', how='outer')

# # Display the resulting DataFrame
# merged_df

In [ ]:
# Split the name_x column into first_name and last_name
def split_name(name):
    parts = name.split()
    if len(parts) > 1:
        return parts[-1], ' '.join(parts[:-1])
    else:
        return parts[0], ''

money_puck_merged[['last_name', 'first_name']] = money_puck_merged['name'].apply(lambda x: pd.Series(split_name(x)))
money_puck_merged.head()

# Combine last_name and first_name into a single column with the format "Last, First"
money_puck_merged['name'] = money_puck_merged['last_name'] + ', ' + money_puck_merged['first_name']

# Drop the temporary first_name and last_name columns
money_puck_merged = money_puck_merged.drop(columns=['first_name', 'last_name'])

# Display the first few rows of the DataFrame to verify the change
len(money_puck_merged)

# Save the merged DataFrame to a CSV file if needed
#money_puck_merged.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/salary/money_puck_merged.csv', index=False)

In [ ]:
matthews = money_puck_merged[money_puck_merged['name'] == 'Crosby, Sidney']
matthews

In [ ]:
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from bs4 import BeautifulSoup
# import time

# # Initialize the Chrome driver
# driver = webdriver.Chrome()

# # Function to get data from a specific page
# def get_page_data():
#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     tables = soup.find_all('table')
#     for table in tables:
#         df = pd.read_html(str(table))[0]
#         df = df.reset_index(drop=True)
#     return df

# # Function to get data from multiple pages
# def get_all_data(total_pages):
#     all_data = pd.DataFrame()
#     driver.get('https://capwages.com/players/active')
    
#     for page in range(1, total_pages + 1):
#         # Wait for the table to load
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.TAG_NAME, 'table'))
#         )
        
#         # Get the data from the page
#         page_data = get_page_data()
#         if page_data is not None:
#             all_data = pd.concat([all_data, page_data], ignore_index=True)
        
#         # Click on the "Next" button to load the next page
#         try:
#             next_button = driver.find_element(By.XPATH, '//button[span[text()="Go to next page"]]')
#             driver.execute_script("arguments[0].click();", next_button)
#             time.sleep(5)  # Wait for the next page to load
#         except Exception as e:
#             print(f"Failed to click on the next button: {e}")
#             break
    
#     return all_data

# # Get data from the first 21 pages
# total_pages = 21
# all_data = get_all_data(total_pages)

# # Save the combined data to a CSV file
# all_data.to_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/salary/cap_all.csv', index=False)

# # Display the combined data
# print(all_data)

# # Close the driver
# driver.quit()

In [ ]:
salaries = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/files/salary/cap_all.csv')
salaries

In [ ]:
salaries = salaries.rename(columns={'PLAYERS': 'name'}).drop(columns=['HAND', 'POS', 'G', 'A', 'PTS', 'SV%', 'TEAM', 'AGE', 'TERM', 'TYPE', 'TOTAL'])
salaries.head()

In [ ]:
salaries_df = pd.merge(merged_skater_result, salaries, on='name', how='inner')
salaries_df

In [ ]:
salaries_df = pd.merge(salaries, merged_goalie_result, on='name', how='outer')
# Check to see if we lost any data
salaries_df

In [ ]:
salaries_df.info()

In [ ]:
# Filter the DataFrame to include only rows where player_id is null
df_null_player_id = salaries_df[salaries_df['player_id'].isnull()]

# Display the first few rows of the filtered DataFrame to verify the change
df_null_player_id.head(25)